### Modelling Covid-19 outbreak in Slovakia 
Miroslav Gasparek, 22 March 2020

In [129]:
import numpy as np

In [223]:
### Parameters
tau = 2/3 # Proportion of time people interact at their home location
alpha = 1 # Mobility parameter
gamma = 0.1 # Inverse duration of infection 

### Initial conditions
S = np.array([10,30,10])
I = np.array([5, 10, 5])
R = np.array([0,0,0])
N = np.array([15, 40, 15])

### Vector of infection rates
beta_vec = np.array([0.5, 0.1, 0.3])

### Mobility matrix 
OD = np.array([[3,5,1], [20,3,5], [2,10,3]])

### Rescaling
x = I / N # Relative No. of susceptibles
y = S / N # Relative No, of infected

### Simulation parameters 
n = 10 # Number of days

# Get the number of municipalities
S_rows = np.shape(S)[0]
I_rows = np.shape(I)[0]
R_rows = np.shape(R)[0]

x_rows = np.shape(x)[0]
y_rows = np.shape(y)[0]


### Pre-allocate the matrices 
S_mat = np.zeros((S_rows, n))
I_mat = np.zeros((I_rows, n))
R_mat = np.zeros((R_rows, n))

x_mat = np.zeros((x_rows, n))
y_mat = np.zeros((y_rows, n))

### Initialize the matrix
S_mat[:,0] = S
I_mat[:,0] = I
R_mat[:,0] = R

In [224]:
### Here we loop over to solve the difference equation
for i in range(n-1):
    
    # Define the second term of the first equation
    first_term = tau*(np.multiply(np.multiply(S_mat[:,i], I_mat[:,i]),beta_vec))/N
    
    # Define the second term
    second_term_1 = alpha*(1-tau)*(S_mat[:,i] - np.multiply(y_mat[:,i],(OD.sum(axis=0) - np.diag(OD)) ))
    second_term_2 = np.matmul(OD,(beta_vec*x_mat[:,i]))-np.diag(OD)*(beta_vec*x_mat[:,i])
    second_term_3 = (I_mat[:,i] - x_mat[:,i]*(OD.sum(axis=0) - np.diag(OD))) * beta_vec
    second_term_den = N - (OD.sum(axis=0) - np.diag(OD)) + (OD.sum(axis=1) - np.diag(OD))
    
    second_term = (second_term_1*(second_term_2+second_term_3))/second_term_den
    
    # Define the third term
    third_term_1 = alpha*(1-tau)*y_mat[:,i]
    third_term_2 = np.matmul((OD.sum(axis=0) - np.diag(OD)), (second_term_2+second_term_3)/second_term_den)
    
    third_term = np.multiply(third_term_1, third_term_2)
    
    # Iterate over the Susceptible
    S_mat[:,i+1] = S_mat[:,i] - first_term - second_term - third_term
    I_mat[:,i+1] = I_mat[:,i] + first_term - gamma*I_mat[:,i] + second_term + third_term
    R_mat[:,i+1] = R_mat[:,i] + gamma*I_mat[:,i]
    
    # Iterate over the normalized values
    x_mat[:,i+1] = I_mat[:,i+1]/N
    y_mat[:,i+1] = S_mat[:,i+1]/N

In [226]:
I_mat

array([[ 5.00000000e+00, -2.72222222e+00, -8.41853797e+00,
        -2.86261247e+01, -1.54743568e+02, -2.82657007e+03,
        -8.12098266e+05, -6.58699506e+10, -4.29358718e+20,
        -1.79632732e+40],
       [ 1.00000000e+01,  9.70000000e+00,  4.50133923e+00,
        -9.49894512e+00, -7.10311752e+01, -8.39451652e+02,
        -1.14311038e+05, -4.17900287e+09, -1.22118000e+19,
        -2.28077794e+38],
       [ 5.00000000e+00,  5.40476190e+00,  1.78428315e+00,
        -1.07870297e+01, -9.44453045e+01, -2.13950440e+03,
        -7.56426646e+05, -7.70726689e+10, -6.47411011e+20,
        -3.61642965e+40]])